In [3]:
import cv2
import numpy as np
import pickle
import cv2
import math


In [4]:
OLS_model = pickle.load(open('OLS.pkl', 'rb'))
SGD_model = pickle.load(open('SGD.pkl', 'rb'))
MeClf_model = pickle.load(open('Meclassifier.pkl', 'rb'))



In [5]:
img_dim = 64

In [6]:
def faceDetect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(img_dim, img_dim)
    )
    
    if(len(faces) > 1):
        cur_max = 0
        face = 0

        for (x, y, w, h) in faces:
            length = math.sqrt((x - (x+w))**2 + (y - (y+h))**2 )
            
            if length > cur_max:
                cur_max = length
                face = [x,y,w,h]

        faces = np.array([face])

    for (x, y, w, h) in faces:
    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw a rectangle around the detected area (face)

        roi_color = image[y+2:y+2 + h-3, x+2:x+2 + w-3] # The pixel area containing the face

        return True,[image,roi_color]
    
    return False,[]

In [14]:
cv2.destroyAllWindows()

cam = cv2.VideoCapture(0)

while True:

    check, frame = cam.read()
    frame = cv2.flip(frame,1)

    
    frame = cv2.copyMakeBorder(frame,1,150,1,1,cv2.BORDER_CONSTANT,value=0)

    cv2.putText(frame,"Who:",(20,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
    cv2.putText(frame,"Age:",(370,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)

    cv2.imshow('video', frame)
    detected,bound_frame = faceDetect(frame)


    if(detected):
        frame = bound_frame[0]
        cropped_frame = bound_frame[1]

        img_resize_clf = cv2.resize(cropped_frame, (64,64))
        img_resize_reg = cv2.resize(cropped_frame, (32,32))

        cur_image_clf = img_resize_clf.flatten()
        cur_image_ref = img_resize_reg.flatten()

        me_pred = MeClf_model.predict([cur_image_clf])
        ols_pred = OLS_model.predict([cur_image_ref])
        sgd_pred = SGD_model.predict([cur_image_ref])

        cv2.putText(frame,"OLS - " + str(int(ols_pred)),(370,580),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"SGD - " + str(int(sgd_pred)),(370,620),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)

        if(me_pred == 0):
            cv2.putText(frame,"Unknown",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

        else:
            cv2.putText(frame,"Sameed",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

    else:
        cv2.putText(frame,"Face Not Detected",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        # cv2.imshow('video', frame)

    cv2.imshow('video', frame)


    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 